## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests

# ----------- Science Jubilee -------------
from science_jubilee import Machine as Jub
from science_jubilee.tools import HTTPSyringe as syringe
from science_jubilee.tools import Pipette, PneumaticSampleLoader
from science_jubilee.utils import Handlers
import time
import numpy as np
import pandas as pd
import logging
import sys
import json
sys.path.append('..')
import stober_synthesis_utils as stober
import usaxs_utils
import uuid

In [4]:
with open('systemconfig.json', 'rt') as f:
    config = json.load(f)

#usaxs_server_ip = config['sample_server']['ip_address']

#afl_ip = '192.168.1.4'

reaction_time = 5*60

In [5]:
FORMAT = '%(asctime)s:%(levelname)s:%(name)s:%(message)s'
logging.basicConfig(filename = 'APS_batchSynthesis_2025_03_29.log', level = logging.INFO, format = FORMAT)
logger = logging.getLogger(__name__)

In [6]:
jubilee = Jub.Machine(address='192.168.1.2', simulated = False, crash_detection = False) 

0
response in connect:  [True, True, True, True]


In [7]:
deck = jubilee.load_deck('lab_automation_deck_AFL_bolton.json')

In [8]:
sample_table = pd.read_csv('Mesoporous_SobolBaseline_APS_BatchMode_3_29_25.csv')
#sample_table_2 = pd.read_csv('SampleTable_ReproReplicates_10000.csv')


In [9]:
sample_table

,Unnamed: 0,sample_name,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,ctab_volume,F127_volume
0,0,SobolBatch_control_1,5dcea3d4-4afc-4389-afbd-03b4640e2860,739.669722,427.068651,1856.721283,1706.732498,4027.093018,1242.714828
1,1,SobolBatch_control_2,5219bd08-7689-483d-92e8-4162d16d8379,739.669722,427.068651,1856.721283,1706.732498,4027.093018,1242.714828
2,2,SobolBatch_control_3,27a11227-cf1f-4da7-8875-26bbc1e11a05,739.669722,427.068651,1856.721283,1706.732498,4027.093018,1242.714828
3,3,SobolBatch_1,4c0400c8-3631-42ca-92f6-6cd1d0f0b0d3,188.708728,536.219853,5902.720493,1899.525156,1395.976302,76.849468
4,4,SobolBatch_2,fe05e774-5c92-4248-8d59-74afda65ee1a,342.046843,412.468670,3856.504701,2496.831857,1821.357608,1070.790321
...,...,...,...,...,...,...,...,...,...
81,81,SobolBatch_76,c60e6fcd-c457-40d8-9d2b-bcf72793a4c1,1114.331675,489.195076,399.276870,2923.317598,4279.141697,794.737084
82,82,SobolBatch_77,bda382f7-2b73-4941-ac9e-9a794d0e2ed3,507.961463,732.685161,2484.911837,3192.106947,2749.532188,332.802404
83,83,SobolBatch_78,0cc6035c-a149-4764-9896-a26aecb380ff,1410.262837,587.635686,734.421605,2377.659058,1454.000166,3436.020646
84,84,SobolBatch_79,f9d13a27-bbf3-4a4b-add5-24313ff8742b,309.837529,172.560101,1203.710347,2253.049331,3830.844133,2229.998561


## Load Tools

In [17]:
syringe_10 = syringe.HTTPSyringe.from_config(2, "../../science-jubilee/src/science_jubilee/tools/configs/10cc_syringe.json")
syringe_1_1 = syringe.HTTPSyringe.from_config(1, "../../science-jubilee/src/science_jubilee/tools/configs/1cc_1_syringe.json")
syringe_1_2 = syringe.HTTPSyringe.from_config(3, "../../science-jubilee/src/science_jubilee/tools/configs/1cc_2_Hamiltonsyringe.json")
syringe_1_3 = syringe.HTTPSyringe.from_config(4, "../../science-jubilee/src/science_jubilee/tools/configs/1cc_3_Hamiltonsyringe.json")
syringe_1_4 = syringe.HTTPSyringe.from_config(0, '../../science-jubilee/src/science_jubilee/tools/configs/1cc_4_syringe.json')

Syringe name:  10cc_1
{'capacity': 10000, 'empty_position': 1830, 'full_position': 1217, 'name': '10cc_1'}
Syringe name:  1cc_1
{'capacity': 1000, 'empty_position': 1845, 'full_position': 1240, 'name': '1cc_1'}
Syringe name:  1cc_2_hamilton
{'capacity': 1000, 'empty_position': 1753, 'full_position': 1120, 'name': '1cc_2_hamilton'}
Syringe name:  1cc_3_hamilton
{'capacity': 1000, 'empty_position': 1753, 'full_position': 1120, 'name': '1cc_3_hamilton'}
Syringe name:  1cc_4
{'capacity': 1000, 'empty_position': 1845, 'full_position': 1240, 'name': '1cc_4'}


In [18]:
jubilee.load_tool(syringe_10)
jubilee.load_tool(syringe_1_1)
jubilee.load_tool(syringe_1_2)
jubilee.load_tool(syringe_1_3)
jubilee.load_tool(syringe_1_4)

In [19]:
mix_syringe = syringe_10
water_syringe = syringe_1_1
ammonia_syringe = syringe_1_2
teos_syringe = syringe_1_3
surfactant_syringe = syringe_1_4

## Load water syringe with water

In [35]:
water_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [36]:
water_syringe.set_pulsewidth(water_syringe.empty_position-1, s = 2000)

In [37]:
water_syringe.load_syringe(0, water_syringe.empty_position-1)

Loaded syringe, remaining volume 0 uL


## Load ammonia syringe

In [38]:
#ammonia_syringe.set_pulsewidth(1420, s = 10)

In [39]:
ammonia_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [40]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.empty_position-1, s = 2000)

In [41]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.full_position+1, s = 10)

In [42]:
ammonia_syringe.load_syringe(850, ammonia_syringe.full_position-1)

Loaded syringe, remaining volume 850 uL


## Load TEOS syringe

In [43]:
teos_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [44]:
teos_syringe.set_pulsewidth(teos_syringe.empty_position-1, s = 2000)

In [45]:
teos_syringe.load_syringe(0, teos_syringe.empty_position - 1)

Loaded syringe, remaining volume 0 uL


## load ethanol syringe

In [66]:
#jubilee.park_tool()

In [46]:
mix_syringe.load_syringe(0, mix_syringe.empty_position-1)

Loaded syringe, remaining volume 0 uL


In [47]:
mix_syringe.set_pulsewidth(mix_syringe.empty_position-1, s = 2000)


## Don't need to load surfactant syringe

In [48]:
surfactant_syringe.set_pulsewidth(surfactant_syringe.empty_position-1)
surfactant_syringe.load_syringe(0, surfactant_syringe.empty_position-1)

Loaded syringe, remaining volume 0 uL


## Load labware


In [49]:
samples_1 = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 2)
samples_1.manual_offset([(30.8, 168.0), (121.3, 168.3), (121.3, 112.6)])

New manual offset applied to 20mlscintillation_12_wellplate_18000ul


In [50]:
samples_2 = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 0)
samples_2.manual_offset([(31.8, 71.0), (121.8, 71.0), (121.8, 16.0)])

New manual offset applied to 20mlscintillation_12_wellplate_18000ul


In [51]:
stocks_1 = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 4)
stocks_1.manual_offset([(30.1, 264.5), (121.1, 264.5), (121.1, 208.5)])

New manual offset applied to 20mlscintillation_12_wellplate_18000ul


In [52]:
stocks_2 = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 4)
stocks_2.manual_offset([(171.1, 264.8), (260.8, 265.0), (260.8, 209.0)])

New manual offset applied to 20mlscintillation_12_wellplate_18000ul


In [53]:
samples = stober.SampleWells({'samples_1':samples_1, 'samples_2':samples_2})

In [54]:
teos_stocks = stober.Stocks([stocks_1[0], stocks_1[1]], usable_volume= 17000)
ammonia_stocks = stober.Stocks([stocks_1[2]], usable_volume= 17000)
water_stocks = stober.Stocks([stocks_1[3], stocks_1[4], stocks_1[5]], usable_volume= 17000)
ethanol_stocks = stober.Stocks([stocks_1[6], stocks_1[7], stocks_1[8], stocks_1[9]], usable_volume = 17000)
f127_stocks = stober.Stocks([stocks_1[10], stocks_1[11], stocks_2[0]], usable_volume = 17000)
ctab_stocks = stober.Stocks([stocks_2[1], stocks_2[2], stocks_2[3], stocks_2[4]], usable_volume= 17000)

rinse_stocks_precursor = [stocks_2[5], stocks_2[6], stocks_2[7]]
rinse_stocks_teos = [stocks_2[8], stocks_2[9], stocks_2[10], stocks_2[11]]

## Check sample alignment

In [28]:
jubilee.deck.safe_z

68.5

In [ ]:
jubilee.pickup_tool(mix_syringe)
for loc in stocks_1:
    jubilee.safe_z_movement()
    jubilee.move_to(x = loc.x, y = loc.y)
    print(loc.y)
    jubilee.move_to(z = loc.top_+7)
    curr_well = loc
    ans = input(f'Is the tip centered over sample {curr_well}?')
    if ans == 'y':
        continue
    else:
        continue
    

In [49]:
sample_table

,Unnamed: 0,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,dilution_volume_fraction,silica_mass_conc,silica_mass_fraction,well
0,0,82c085e1-a523-40ce-9e89-4fa0bf546525,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,B2
1,1,747534b1-7370-49a5-bdc1-4fce5227f391,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,B3
2,2,e5f6e82f-0282-46c8-be77-d8914adca9b5,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,B4
3,3,c87ac840-ddde-43ab-a8e3-b5e3eb5a273d,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,C1
4,4,d034d82c-e329-484c-91cc-75ea655a1446,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,C2


In [34]:
jubilee.park_tool()


In [106]:
loc = samples['A8']
jubilee.move_to(x = loc.x, y = loc.y)
print(loc.y)
jubilee.move_to(z = loc.top_+7)

175.8614659806774


# Experiment


In [241]:
jubilee.park_tool()

## add ethanol

In [58]:
sample_table_batch1 = sample_table.iloc[1:24]

In [59]:
sample_table_batch1

,Unnamed: 0,sample_name,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,ctab_volume,F127_volume
1,1,SobolBatch_control_2,5219bd08-7689-483d-92e8-4162d16d8379,739.669722,427.068651,1856.721283,1706.732498,4027.093018,1242.714828
2,2,SobolBatch_control_3,27a11227-cf1f-4da7-8875-26bbc1e11a05,739.669722,427.068651,1856.721283,1706.732498,4027.093018,1242.714828
3,3,SobolBatch_1,4c0400c8-3631-42ca-92f6-6cd1d0f0b0d3,188.708728,536.219853,5902.720493,1899.525156,1395.976302,76.849468
4,4,SobolBatch_2,fe05e774-5c92-4248-8d59-74afda65ee1a,342.046843,412.468670,3856.504701,2496.831857,1821.357608,1070.790321
5,5,SobolBatch_3,aab54b4e-1d8c-4262-a158-e9cdf14edbff,1279.257002,197.989525,2256.955528,1727.886502,4074.655569,463.255875
6,6,SobolBatch_4,51558b9d-5018-4c0c-bca8-16b264207ef9,1379.696090,476.024321,1401.834644,2942.551750,1471.099903,2328.793293
7,7,SobolBatch_5,ae8570a4-85c6-4903-bfa6-852670463b4d,1533.021926,374.232377,2899.190459,2423.655953,948.718541,1821.180744
8,8,SobolBatch_6,480d0a24-41e8-4285-b3b3-a7ea95ca0144,1025.290547,663.538793,194.788540,1534.896377,5147.818158,1433.667585
9,9,SobolBatch_7,eb9910af-e02a-4fa9-9c48-ca43fcb17d67,797.951488,386.142736,2583.533393,3290.473863,2696.846612,245.051907
10,10,SobolBatch_8,df8f9bd5-efa9-4625-befc-6d481feac9e0,550.311928,464.820919,1536.394183,3093.254740,3497.176443,858.041788


In [60]:
for i, row in sample_table_batch1.iterrows():


    if i %3 == 0 and i !=0:
        # swap out CTAB
        result = input('CTAB swap out required. swap ctab with fresh from the hot plate.')
        
    sample_uid = row['uuid']
    sample_name = row['sample_name']
    well = samples.get_well_for_sample(sample_uid)
    teos_volume = row['teos_volume']
    ethanol_volume = row['ethanol_volume']
    ctab_volume = row['ctab_volume']
    f127_volume = row['F127_volume']

    sample_table_select = sample_table.iloc[i:i+1]
    location_lookup_selected = {sample_uid:well}



    start_time = time.time()
    logger.info(f'Starting sample {sample_name}, {sample_uid} in well {well}')
    print(f'Starting sample {sample_name}, {sample_uid} in well {well}')
    #add ethanol
    stober.reactant_transfer(jubilee, mix_syringe, ethanol_stocks, well, ethanol_volume, 500)
    logger.info(f'Added {ethanol_volume} uL ethanol to {str(sample_uid)}') 
    
    #add water
    stober.add_reactants_batch(jubilee, water_syringe, mix_syringe, sample_table_select, location_lookup_selected, 'water_volume', water_stocks, dwell_time = 7)
    print(f'added water')
    logger.info(f'Added water to {str(sample_uid)}') 
    #add ammonia
    stober.add_reactants_batch(jubilee, ammonia_syringe, mix_syringe, sample_table_select, location_lookup_selected, 'ammonia_volume', ammonia_stocks, dwell_time = 10, refill_dwell=5)
    print('added ammonia')
    logger.info(f'Added ammonia to {str(sample_uid)}') 
    # add F127
    
    stober.reactant_transfer(jubilee, surfactant_syringe, f127_stocks, well, f127_volume, 50, rinse_stocks=rinse_stocks_precursor)
    print(f'added {f127_volume} f127')
    logger.info(f'Added {f127_volume} uL F127 to {str(sample_uid)}') 
    
    stober.reactant_transfer(jubilee, surfactant_syringe, ctab_stocks, well, ctab_volume, 50, rinse_stocks=rinse_stocks_precursor)
    print(f'added {ctab_volume} uL CTAB')
    logger.info(f'Added {ctab_volume} uL ctab to {str(sample_uid)}') 
    # mix precursors
    
    stober.first_mix(jubilee, mix_syringe, 5000, location_lookup_selected, rinse_stocks_precursor, 3, n_rinse = 1)
    print('mixed samples')
    # add teos
    teos_add_time = stober.add_reactants_batch(jubilee, teos_syringe, mix_syringe, sample_table_select, location_lookup_selected, 'teos_volume', teos_stocks, mix_after=(8000, 5, rinse_stocks_teos), wait = False, n_rinse = 1, return_time = True)
    
    logger.info(f'Added TEOS to sample {sample_name} at {time.strftime("%H:%M", time.localtime(teos_add_time))}')
    
    print(f'Time from start to TEOS addition: {teos_add_time - start_time} s')
    
    print(f'########## Complete sample {sample_name} ############')




Starting sample SobolBatch_control_2, 5219bd08-7689-483d-92e8-4162d16d8379 in well Well A2 form 20mLscintillation 12 Well Plate 18000 µL on slot 2
Loaded syringe, remaining volume 0 uL
breaking dispense into 1 of volume 1706.73249814054
Loaded syringe, remaining volume 0 uL
breaking dispense into 3 of volume 618.9070943256633
remaining vol:  949.0


KeyboardInterrupt: 